<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/challenges/clock-decomposition/notebook-banner.jpg?inline=false" alt="Drawing" style="height: 400px;"/></p>

# What is the notebook about?

The challenge is to use the features extracted from the Clock Drawing Test to build an automated and algorithm to predict whether each participant is one of three phases:

1)    Pre-Alzheimer’s (Early Warning)
2)    Post-Alzheimer’s (Detection)
3)    Normal (Not an Alzheimer’s patient)

In machine learning terms: this is a 3-class classification task.

# How to use this notebook? 📝

<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/notebook/aicrowd_notebook_submission_flow.png?inline=false" alt="notebook overview" style="width: 650px;"/></p>

- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_DATASET_PATH` | Path to the file containing test data (The data will be available at `/ds_shared_drive/` on aridhia workspace). This should be an absolute path.
`AICROWD_PREDICTIONS_PATH` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#install-packages-) section to install the packages
- **Training your models**. All the code within the [Training phase ⚙️](#training-phase-) section will be skipped during evaluation. **Please make sure to save your model weights in the assets directory and load them in the predictions phase section** 

# Setup AIcrowd Utilities 🛠

We use this to bundle the files for submission and create a submission on AIcrowd. Do not edit this block.

In [1]:
!pip install -q -U aicrowd-cli

In [2]:
%load_ext aicrowd.magic

# AIcrowd Runtime Configuration 🧷

Define configuration parameters. Please include any files needed for the notebook to run under `ASSETS_DIR`. We will copy the contents of this directory to your final submission file 🙂

The dataset is available under `/ds_shared_drive` on the workspace.

In [3]:
import os

# Please use the absolute for the location of the dataset.
# Or you can use relative path with `os.getcwd() + "test_data/validation.csv"`
AICROWD_DATASET_PATH = os.getenv("DATASET_PATH", "/ds_shared_drive/validation.csv")
AICROWD_PREDICTIONS_PATH = os.getenv("PREDICTIONS_PATH", "predictions.csv")
AICROWD_ASSETS_DIR = "assets"


# Install packages 🗃

Please add all pacakage installations in this section

In [4]:
!pip install -q numpy pandas

In [5]:
!pip install -q -U fastcore fastai

In [6]:
!pip install -q torch

In [7]:
!pip install -q optuna

In [8]:
!pip install -q seaborn lightgbm scikit-learn

In [9]:
!pip install xgboost

In [45]:
!pip install -q scipy

# Define preprocessing code 💻

The code that is common between the training and the prediction sections should be defined here. During evaluation, we completely skip the training section. Please make sure to add any common logic between the training and prediction sections here.

### Import common packages

Please import packages that are common for training and prediction phases here.

In [46]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.neighbors import NearestCentroid, LocalOutlierFactor, KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from fastai.tabular.all import *
from sklearn.utils import compute_class_weight
import xgboost as xgb
import torch
import optuna
import joblib
from scipy.stats import mode

In [11]:
train_data = pd.read_csv(os.getenv("DATASET_PATH", "/ds_shared_drive/train.csv"))
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32777 entries, 0 to 32776
Columns: 122 entries, row_id to diagnosis
dtypes: float64(111), int64(8), object(3)
memory usage: 30.5+ MB


# Training phase ⚙️

You can define your training code here. This sections will be skipped during evaluation.

In [12]:
# train_data = train_data[train_data.diagnosis=='normal'].sample(6000).append(train_data[train_data.diagnosis!='normal'])
train_data.diagnosis.value_counts()

normal            31208
post_alzheimer     1149
pre_alzheimer       420
Name: diagnosis, dtype: int64

## Load training data

In [13]:
# class_wts = compute_class_weight(class_weight='balanced', 
#                                 classes=['normal','post_alzheimer', 'pre_alzheimer'],y=train_data.diagnosis)
# class_wts = torch.from_numpy(class_wts)
class_wts = torch.FloatTensor([0.3501,  9.5088, 26.0135])
# class_wts

In [14]:
#train_data = train_data[train_data.diagnosis.isin(['post_alzheimer','pre_alzheimer'])]\
# .append(train_data[train_data.diagnosis.isin(['normal'])].sample(6000))
#train_data.info()

In [15]:
'''interaction_generator = PolynomialFeatures(degree=2, interaction_only=True)
train_interactions = interaction_generator.fit_transform(train_data.fillna(0).drop(columns=['row_id', 
                                                                                  'intersection_pos_rel_centre', 
                                                                                  'diagnosis',
                                                                                  'actual_hour_digit',
                                                                                  'actual_minute_digit']))
train_interactions = pd.DataFrame(train_interactions)
train_interactions.columns = interaction_generator.get_feature_names()
train_interactions.info()
train_data = pd.concat([train_data, train_interactions], axis=1)
train_data.info()'''

"interaction_generator = PolynomialFeatures(degree=2, interaction_only=True)\ntrain_interactions = interaction_generator.fit_transform(train_data.fillna(0).drop(columns=['row_id', \n                                                                                  'intersection_pos_rel_centre', \n                                                                                  'diagnosis',\n                                                                                  'actual_hour_digit',\n                                                                                  'actual_minute_digit']))\ntrain_interactions = pd.DataFrame(train_interactions)\ntrain_interactions.columns = interaction_generator.get_feature_names()\ntrain_interactions.info()\ntrain_data = pd.concat([train_data, train_interactions], axis=1)\ntrain_data.info()"

In [16]:
dep_var = 'diagnosis'
cont_var = train_data.drop(columns=['row_id', 'intersection_pos_rel_centre', 
                                    'diagnosis',
                                    'actual_hour_digit',
                                    'actual_minute_digit'
                                   ]).columns.tolist() 
cat_var = ['intersection_pos_rel_centre']

procs = [FillMissing, Categorify, Normalize]

kfold = KFold(n_splits=5, random_state=42, shuffle=True)

val_data = pd.read_csv(os.getenv("DATASET_PATH", "/ds_shared_drive/validation.csv"))
val_actuals = pd.read_csv(os.getenv("DATASET_PATH", "/ds_shared_drive/validation_ground_truth.csv"))
val_merge = val_data.merge(val_actuals, on=['row_id'])
train_data_full = train_data.append(val_merge[train_data.columns])
train_data_full['iprc'] = train_data_full['intersection_pos_rel_centre']

iprc_num = {'iprc':{'TL':4, 'BL':3, 'TR':2, 'BR':1}}
train_data_full.replace(iprc_num, inplace=True)
train_data.replace(iprc_num, inplace=True)
val_merge['iprc'] = val_merge['intersection_pos_rel_centre']
val_merge.replace(iprc_num, inplace=True)

target_col = "diagnosis"
target_values = ["normal", "post_alzheimer", "pre_alzheimer"]
seed=2021
'''df_pos = train_data_full[train_data_full[target_col].isin(target_values[1:])]
nb_pos = df_pos.shape[0]
nb_neg = nb_pos
df_neg = train_data_full[train_data_full[target_col] == "normal"].sample(n=nb_neg, random_state=seed)
df_samples = pd.concat([df_pos, df_neg]).sample(frac=1).reset_index(drop=True)
train_data_full = df_samples.copy(deep=True)'''



train_data_full.shape, train_data.shape

((33139, 123), (32777, 122))

In [17]:
train_data_full['intersection_pos_rel_centre'].value_counts()

TL    10362
BL     5283
TR     4961
BR     2467
Name: intersection_pos_rel_centre, dtype: int64

## Train your model

In [18]:
nc = NearestCentroid()
rs = RobustScaler()
knn = KNeighborsClassifier(n_neighbors=5)

nc.fit(train_data_full.drop(columns=['row_id',
                                     'intersection_pos_rel_centre',
                                     'diagnosis']).fillna(-1), train_data_full['diagnosis'])
train_data_full['nc'] = nc.predict(train_data_full.drop(columns=['row_id',
                                     'intersection_pos_rel_centre',
                                     'diagnosis']).fillna(-1)).reshape((-1,1))


nc_num = {'nc':{'normal':1, 'pre_alzheimer':2, 'post_alzheimer':3}, 
          'knn':{'normal':1, 'pre_alzheimer':2, 'post_alzheimer':3}}

train_data_full.replace(nc_num, inplace=True)

train_data_full.shape

(33139, 124)

In [19]:
poly_feats = PolynomialFeatures(degree=2, interaction_only=True)
X_train_int = poly_feats.fit_transform(train_data_full.drop(columns=['row_id',
                                                                     'intersection_pos_rel_centre',
                                                                     'diagnosis']).fillna(-1))
X_train_int.shape

(33139, 7382)

In [20]:
rf_poly = RandomForestClassifier(n_estimators=200)
rf_poly.fit(X_train_int, train_data_full['diagnosis'])
importances = rf_poly.feature_importances_
importances

array([0.00000000e+00, 2.51335634e-04, 6.37351371e-05, ...,
       2.29101112e-05, 3.03049883e-05, 4.16668412e-05])

In [21]:
int_index = (-importances).argsort()[:500]
X_poly = X_train_int[:,int_index]
df_poly = pd.DataFrame(X_poly)
df_poly.columns = [poly_feats.get_feature_names()[i] for i in int_index]
df_poly.shape

(33139, 500)

In [22]:
train_data_full = pd.concat([train_data_full.reset_index(drop=True), df_poly.reset_index(drop=True)], axis=1)
train_data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33139 entries, 0 to 33138
Columns: 624 entries, row_id to x75 x90
dtypes: float64(612), int64(9), object(3)
memory usage: 157.8+ MB


In [23]:
train_data_all_in = train_data_full.copy(deep=True)
train_data_full, val_merge =  train_test_split(train_data_all_in, test_size=0.2, random_state=42)
print(train_data_full.shape, val_merge.shape)
train_data_full.diagnosis.value_counts(), val_merge.diagnosis.value_counts()

(26511, 624) (6628, 624)


(normal            25210
 post_alzheimer      958
 pre_alzheimer       343
 Name: diagnosis, dtype: int64,
 normal            6268
 post_alzheimer     253
 pre_alzheimer      107
 Name: diagnosis, dtype: int64)

In [24]:
train_data_full = train_data_full.fillna(-1)
val_merge = val_merge.fillna(-1)
train_data_full.shape, val_merge.shape

((26511, 624), (6628, 624))

In [25]:
le = LabelEncoder()
le.fit(train_data_full.diagnosis)

y_train = le.transform(train_data_full.diagnosis)
y_test = le.transform(val_merge.diagnosis)
dtrain = xgb.DMatrix(train_data_full.drop(columns=['row_id','intersection_pos_rel_centre','diagnosis']),
                                                   y_train)
dtest = xgb.DMatrix(val_merge.drop(columns=['row_id','intersection_pos_rel_centre','diagnosis']),
                                            y_test)


In [ ]:

def objective(trial: optuna.Trial):
    param = {
        "num_class":3,
        "silent":1, 
        "objective":"multi:softmax",
        "eval_metric":"mlogloss",
        "booster":trial.suggest_categorical("booster",['gbtree','gblinear','dart']),
        "lambda":trial.suggest_loguniform("lambda", 1e-8, 1.0), 
        "alpha":trial.suggest_loguniform("alpha", 1e-8, 1.0)
    }

    if((param['booster']=="gbtree") or (param['booster']=='gblinear')):
        param['max_depth'] = trial.suggest_int('max_depth', 1, 9)
        param['eta'] = trial.suggest_loguniform('eta', 1e-8, 1.0)
        param['gamma'] = trial.suggest_loguniform('gamma', 1e-8, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy',['depthwise','lossguide'])
        
    if(param['booster']=='dart'):
        param['sample_type'] = trial.suggest_categorical('sample_type',['uniform','weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type',['tree','forest'])
        param['rate_drop'] = trial.suggest_loguniform('rate_drop',1e-8, 1.0)
        param['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-mlogloss")
    bst = xgb.train(param, dtrain, evals=[(dtest, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dtest)
    
    pred_1 = []
    for pred in preds:
        if(pred==0):
            pred = [0.6, 0.2, 0.2]
        elif(pred==1):
            pred = [0.2, 0.6, 0.2]
        else:
            pred = [0.2, 0.2, 0.6]
        pred_1.append(pred)
    preds = np.array(pred_1)
    # preds.shape
    
    per_ = log_loss(val_merge['diagnosis'].values, preds)
    return per_

study = optuna.create_study()
study.optimize(objective)
best_trial = study.best_trial


In [ ]:
# best_trial

In [ ]:
#  {'n_layers': 4, 'n_epochs': 3, 'batch_size': 32, 'num_units_layer_0': 1000, 'dropout_p_layer_0': 0.9, 'num_units_layer_1': 1100, 'dropout_p_layer_1': 0.35000000000000003, 'num_units_layer_2': 800, 'dropout_p_layer_2': 0.6000000000000001, 'emb_drop': 0.5}. 


In [ ]:
#  Best is trial 9 with value: 0.5676789754577772.
best_parameters = {'booster': 'dart', 
                   'lambda': 0.6718751470224827, 
                   'alpha': 0.01667738524563801, 
                   'sample_type': 'uniform', 
                   'normalize_type': 'tree', 
                   'rate_drop': 0.003246725607740222, 
                   'skip_drop': 6.0236365624270305e-06}

In [26]:
param = {
        "num_class":3,
        "silent":1, 
        "objective":"multi:softmax",
        "eval_metric":"mlogloss",
        'booster': 'dart', 
    'lambda': 0.6718751470224827, 
    'alpha': 0.01667738524563801, 
    'sample_type': 'uniform', 
    'normalize_type': 'tree', 
    'rate_drop': 0.003246725607740222, 
    'skip_drop': 6.0236365624270305e-06
    }

#y_train1 = le.transform(train_data_all_in.diagnosis)
#dtrain1 = xgb.DMatrix(train_data_all_in.drop(columns=['row_id','intersection_pos_rel_centre','diagnosis']),
#                                                   y_train1)

#bst = xgb.train(param, dtrain1
#                , evals=[(dtest, "validation")]
#               )
    

In [30]:
skf = StratifiedKFold(n_splits=5, random_state=2021, shuffle=True)

le = LabelEncoder()
le.fit(train_data_full.diagnosis)

clfs = []
for fold, (itrain, ivalid) in enumerate(skf.split(train_data_full, train_data_full.diagnosis)):
    print("-"*40)
    print(f"Running for fold {fold}")
    
    y_train = le.transform(train_data_all_in.iloc[itrain,:].diagnosis)
    y_test = le.transform(train_data_all_in.iloc[ivalid,:].diagnosis)
    dtrain = xgb.DMatrix(train_data_all_in.iloc[itrain,:].drop(columns=['row_id','intersection_pos_rel_centre','diagnosis']),
                                                       y_train)
    dtest = xgb.DMatrix(train_data_all_in.iloc[ivalid,:].drop(columns=['row_id','intersection_pos_rel_centre','diagnosis']),
                                                y_test)
    
    bst = xgb.train(param, dtrain, evals=[(dtest, "validation")])
    clfs.append(bst)

len(clfs)

----------------------------------------
Running for fold 0
[04:57:38] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-mlogloss:0.75256
[1]	validation-mlogloss:0.55988
[2]	validation-mlogloss:0.43791
[3]	validation-mlogloss:0.35738
[4]	validation-mlogloss:0.30283
[5]	validation-mlogloss:0.26516
[6]	validation-mlogloss:0.23956
[7]	validation-mlogloss:0.22189
[8]	validation-mlogloss:0.21000
[9]	validation-mlogloss:0.20150
----------------------------------------
Running for fold 1
[04:58:03] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are

5

## Save your trained model

In [31]:
i=0
for clf in clfs:
    joblib.dump(bst,f'{AICROWD_ASSETS_DIR}/model_'+str(i)+'.pkl')
    i=i+1

joblib.dump(le, f'{AICROWD_ASSETS_DIR}/le.pkl')

poly_filename = f'{AICROWD_ASSETS_DIR}/poly_feats.pkl'
joblib.dump(poly_feats, poly_filename)

nc_filename = f'{AICROWD_ASSETS_DIR}/nc_model.pkl'
joblib.dump(nc, nc_filename)

nc_num_filename  = f'{AICROWD_ASSETS_DIR}/nc_num.pkl'
joblib.dump(nc_num, nc_num_filename)

rs_filename = f'{AICROWD_ASSETS_DIR}/rs.pkl'
joblib.dump(rs, rs_filename)

iprc_filename = f'{AICROWD_ASSETS_DIR}/iprc.pkl'
joblib.dump(iprc_num, iprc_filename)

meta = {
    "inter_index": int_index
}
meta_filename = f'{AICROWD_ASSETS_DIR}/model_lgb_meta.pkl'
joblib.dump(meta, meta_filename)

['assets/model_lgb_meta.pkl']

# Prediction phase 🔎

Please make sure to save the weights from the training section in your assets directory and load them in this section

In [32]:
# model = load_model_from_assets_dir(AIAICROWD_ASSETS_DIR)
# learners = [load_learner(AICROWD_ASSETS_DIR+'/fastai_tabular'+str(i)+'.mdl') for i in range(5)]
# len(learners)
clfs = []
n = 5

for i in range(n):
    clfs.append(joblib.load(f'{AICROWD_ASSETS_DIR}/model_'+str(i)+'.pkl'))
print(clfs)
# bst = joblib.load(AICROWD_ASSETS_DIR+'/model.pkl')
le = joblib.load(AICROWD_ASSETS_DIR+'/le.pkl')

meta_filename = f'{AICROWD_ASSETS_DIR}/model_lgb_meta.pkl'
meta = joblib.load(meta_filename)
print(meta.keys())

poly_filename = f'{AICROWD_ASSETS_DIR}/poly_feats.pkl'
poly_feats = joblib.load(poly_filename)

nc_filename = f'{AICROWD_ASSETS_DIR}/nc_model.pkl'
nc = joblib.load(nc_filename)

nc_num_filename  = f'{AICROWD_ASSETS_DIR}/nc_num.pkl'
nc_num = joblib.load(nc_num_filename)

rs_filename = f'{AICROWD_ASSETS_DIR}/rs.pkl'
rs = joblib.load(rs_filename)

iprc_filename = f'{AICROWD_ASSETS_DIR}/iprc.pkl'
iprc_num = joblib.load(iprc_filename)


int_index = meta['inter_index']

[<xgboost.core.Booster object at 0x7f9fbbaabb50>, <xgboost.core.Booster object at 0x7f9fbbaab910>, <xgboost.core.Booster object at 0x7f9fbbaab3a0>, <xgboost.core.Booster object at 0x7f9fbbaabdc0>, <xgboost.core.Booster object at 0x7f9fbbaabc70>]
dict_keys(['inter_index'])


In [33]:
len(clfs)

5

## Load test data

In [34]:
test_data = pd.read_csv(AICROWD_DATASET_PATH)
test_data.head()

,row_id,number_of_digits,missing_digit_1,missing_digit_2,missing_digit_3,missing_digit_4,missing_digit_5,missing_digit_6,missing_digit_7,missing_digit_8,...,top_area_perc,bottom_area_perc,left_area_perc,right_area_perc,hor_count,vert_count,eleven_ten_error,other_error,time_diff,centre_dot_detect
0,LA9JQ1JZMJ9D2MBZV,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500272,0.499368,0.553194,0.446447,0,0,0,1,NaN,NaN
1,PSSRCWAPTAG72A1NT,6.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.572472,0.427196,0.496352,0.503273,0,1,0,1,NaN,NaN
2,GCTODIZJB42VCBZRZ,11.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.494076,0.505583,0.503047,0.496615,1,0,0,0,0.0,0.0
3,7YMVQGV1CDB1WZFNE,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.555033,0.444633,0.580023,0.419575,0,1,0,1,NaN,NaN
4,PHEQC6DV3LTFJYIJU,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.603666,0.395976,0.494990,0.504604,0,0,0,1,150.0,0.0


## Generate predictions

In [35]:
'''test_rs = rs.fit_transform(test_data.drop(columns=['row_id',
                                                  'intersection_pos_rel_centre']).fillna(-1))
test_data['nc'] = nc.predict(test_rs)'''

test_data['iprc'] = test_data['intersection_pos_rel_centre']
test_data.replace(iprc_num, inplace=True)

test_data['nc'] = nc.predict(test_data.drop(columns=['row_id',
                                     'intersection_pos_rel_centre']).fillna(-1)).reshape((-1,1))

test_data.replace(nc_num, inplace=True)


X_test_int = poly_feats.transform(test_data.drop(columns=['row_id',
                                                         'intersection_pos_rel_centre']).fillna(-1))
X_poly_test = X_test_int[:,int_index]
df_poly_test = pd.DataFrame(X_poly_test)
df_poly_test.columns = [poly_feats.get_feature_names()[i] for i in int_index]
test_data = pd.concat([test_data, df_poly_test], axis=1)
test_data.shape

(362, 623)

In [68]:
dsub = xgb.DMatrix(test_data.drop(columns=['row_id','intersection_pos_rel_centre']))

prs = []
for clf in clfs:
    prs.append(clf.predict(dsub))
prs = np.array(prs).T
# preds = mode(prs, axis=1)[0]
preds = prs.mean(axis=1)

In [69]:
pred_1 = []
for pred in list(preds):
    if(pred<0.5):
        pred = [0.6, 0.2, 0.2]
    elif(pred<1.5):
        pred = [0.2, 0.6, 0.2]
    else:
        pred = [0.2, 0.2, 0.6]
    pred_1.append(pred)
preds = np.array(pred_1)
preds.shape

(362, 3)

In [70]:
predictions = {
    "row_id": test_data["row_id"].values,
    "normal_diagnosis_probability": preds[:,0],
    "post_alzheimer_diagnosis_probability": preds[:,1],
    "pre_alzheimer_diagnosis_probability": preds[:,2],
}

predictions_df = pd.DataFrame.from_dict(predictions)

## Save predictions 📨

In [71]:
predictions_df.to_csv(AICROWD_PREDICTIONS_PATH, index=False)

# Submit to AIcrowd 🚀

**NOTE: PLEASE SAVE THE NOTEBOOK BEFORE SUBMITTING IT (Ctrl + S)**

In [67]:
!DATASET_PATH=$AICROWD_DATASET_PATH \
aicrowd notebook submit \
    --assets-dir $AICROWD_ASSETS_DIR \
    --challenge addi-alzheimers-detection-challenge

API Key valid
Saved API Key successfully!
Using notebook: /home/desktop0/python-optuna-xgboost.ipynb for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...
Validating the submission...
Executing install.ipynb...
[NbConvertApp] Converting notebook /home/desktop0/submission/install.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python
[NbConvertApp] Writing 4234 bytes to /home/desktop0/submission/install.nbconvert.ipynb
Executing predict.ipynb...
[NbConvertApp] Converting notebook /home/desktop0/submission/predict.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python
[NbConvertApp] Writing 24283 bytes to /home/desktop0/submission/predict.nbconvert.ipynb
submission.zip ━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 76.0/76.0 MB • 2.4 MB/s • 0:00:00 • 0:00:010:00:02
                                                 ╭─────────────────────────╮                                                 
                        